In [1]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import math
import copy as cp
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import catboost as cbt
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss
import gc
import math
import time
from tqdm import tqdm
import datetime
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from datetime import datetime,timedelta
import warnings
import os
import statsmodels.api as sm
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from sklearn.model_selection import train_test_split
import copy as cp
import xgboost as xgb

In [2]:
train_1 = pd.read_csv('../data_3/second_round_training_data.csv')[['Parameter'+str(i) for i in range(1,11)]+['Quality_label']]
# test = pd.read_csv('../data_3/second_round_testing_data.csv')
# submit = pd.read_csv('../data_3/submit_example2.csv')

In [3]:
train_2 = pd.read_csv('../data/first_round_training_data.csv')[['Parameter'+str(i) for i in range(1,11)]+['Quality_label']]

In [4]:
train = pd.concat([train_1,train_2],sort = False)
test = pd.read_csv('../data_3/second_round_testing_data_2.csv')
submit = pd.read_csv('../data_3/submit_example2.csv')

#train = train.drop_duplicates(['Parameter5','Parameter7','Parameter8','Parameter9','Parameter10','Attribute4','Attribute5','Attribute6','Attribute7','Attribute8','Attribute9','Attribute10'],'first')
#来做特征啦啦
data = train.append(test,sort=False).reset_index(drop=True)
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
data['label'] = data['Quality_label'].map(dit)
train['label'] = train['Quality_label'].map(dit)
del data['Quality_label']
data = data.fillna(-1)


# data['moby6'] = data['Parameter5']/(data['Parameter6']+data['Parameter7']+data['Parameter8']+data['Parameter9']+data['Parameter10'])#太完美了
# data['moby7'] = (data['Parameter5']+data['Parameter6'])/(data['Parameter7']+data['Parameter8']+data['Parameter9']+data['Parameter10'])
# data['moby8'] = data['Parameter8']/(data['Parameter5']+data['Parameter6']+data['Parameter7']+data['Parameter8']+data['Parameter9']+data['Parameter10'])

#略有用，重复度计数
# m = data.groupby(['Parameter6'])['label'].count().reset_index(name='count6')
# data = pd.merge(data,m,on ='Parameter6',how = 'left' )

# m = data.groupby(['Parameter7'])['label'].count().reset_index(name='count7')
# data = pd.merge(data,m,on ='Parameter7',how = 'left' )
# m = data.groupby(['Parameter8'])['label'].count().reset_index(name='count8')
# data = pd.merge(data,m,on ='Parameter8',how = 'left' )
# # m = data.groupby(['Parameter9'])['label'].count().reset_index(name='count9')
# # data = data.merge(m)
# m = data.groupby(['Parameter10'])['label'].count().reset_index(name='count10')
# data = pd.merge(data,m,on ='Parameter10',how = 'left' )

# m = data.groupby(['Parameter10'])['label'].mean().rank().reset_index(name='rank')
# data = pd.merge(data,m,on ='Parameter10',how = 'left' )

# data['moby1'] = data['Parameter7']/(data['Parameter5']+data['Parameter6']+data['Parameter8'])
# # data['moby2'] = data['Parameter10']/(data['Parameter5']+data['Parameter6']+data['Parameter8']+data['Parameter9'])
# # data['moby3'] = data['Parameter8']/(data['Parameter5']+data['Parameter6']+data['Parameter7']+data['Parameter8']+data['Parameter9']+data['Parameter10'])
# # data['moby6'] = (data['Parameter5']+data['Parameter6'])/(data['Parameter9'])
# #0.6985


# # data['moby11'] =data['Parameter5']/data['Parameter9']  0.6971
# # data['moby13'] =data['Parameter6']/data['Parameter5']  0.6966
# # data['moby14'] =data['Parameter6']/data['Parameter7']  0.684
# # data['moby16'] =data['Parameter6']/data['Parameter9']  
# # 0.69966 目前最高


# data['moby20'] =data['Parameter7']/data['Parameter8']

# # data['moby21'] =data['Parameter7']/data['Parameter9']

# # data['p5_6'] = data['Parameter5'].astype(str)+'_'+data['Parameter6'].astype(str)
# # lbl = LabelEncoder()
# # lbl.fit(list(data['p5_6'].values))
# # data['p5_6'] = lbl.transform(list(data['p5_6'].values))


data = data.fillna(-1)
train = data.loc[data.label>=0]
test = data.loc[data.label<0]
# test = test.sort_values(by=['Group'], ascending=True)
label = train['label']
# feature = [f for f in data.columns if f not in ['label','Group','Parameter1','Parameter2','Parameter3','Parameter4']]
feature = [f for f in data.columns if f not in ['label','Group']]

train_x = train[feature]
train_y = train['label'].astype(np.int32)

test_id = test['Group']
test_x = test[feature].reset_index(drop = True)


In [5]:
train_x.head()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10
0,0.167898,104.443982,2.772825,0.146548,0.000421,0.000612,2286.523413,0.035407,0.593081,1.010385
1,252.483066,0.343232,0.066873,0.002495,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385
2,4.124654,0.170534,0.383800,0.000004,0.000909,0.001972,2286.523413,0.035407,0.593081,1.010385
3,294.656750,6.153711,0.014716,4284.326273,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385
4,0.026284,0.166810,7.587398,0.002202,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385


In [6]:
test_x.head()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10
0,0.007218,5.081092,1.132919,931.721241,2.225808,2.208755,0.038483,2.931083,6.783967,1.010385
1,0.527218,0.025262,1.049852,0.050764,4.803947,3.965023,0.038483,2.931083,6.783967,0.377332
2,0.004863,85.700444,4.002861,1.443409,4.803947,3.965023,0.038483,2.931083,6.783967,0.377332
3,5.387838,2.456417,8.820446,0.408974,2.484379,3.965023,0.038483,2.931083,6.783967,0.377332
4,3.355337,0.076572,23.199547,1.172201,3.095123,2.684398,0.038483,2.931083,2.005852,0.377332


In [7]:
train_x['Parameter1'] = np.log(train_x['Parameter1']+1e-05).round(3)
train_x['Parameter2'] = np.log(train_x['Parameter2']+1e-05).round(3)
train_x['Parameter3'] = np.log(train_x['Parameter3']+1e-05).round(3)
train_x['Parameter4'] = np.log(train_x['Parameter4']+1e-05).round(3)


test_x['Parameter1'] = np.log(test_x['Parameter1']+1e-05).round(3)
test_x['Parameter2'] = np.log(test_x['Parameter2']+1e-05).round(3)
test_x['Parameter3'] = np.log(test_x['Parameter3']+1e-05).round(3)
test_x['Parameter4'] = np.log(test_x['Parameter4']+1e-05).round(3)

# xgb

# 交叉验证ｘｇｂ

In [7]:
tmp_pre = np.zeros((len(test_x),4))
sub1 = pd.DataFrame()


folds = KFold(n_splits=5, shuffle=False, random_state=2019)
oof = np.zeros([train_x.shape[0],4])
predictions = np.zeros([test_x.shape[0],4])

param = {
        'booster': 'gbtree',
        'objective': 'multi:softprob',  # 多分类的问题
        'num_class': 4,               # 类别数，与 multisoftmax 并用
        'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
        'max_depth': 12,               # 构建树的深度，越大越容易过拟合
        'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
        'subsample': 0.9,              # 随机采样训练样本
        'colsample_bytree': 0.9,       # 生成树时进行的列采样
        'min_child_weight': 3,
        'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
        'eta': 0.007,                  # 如同学习率
        'seed': 112211,
        'nthread': 6,                  # cpu 线程数
    }

# test_x = xgb.DMatrix(test_x)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_x, train_y)):
        print("fold n {}".format(fold_+1))
        
        tr_x,tr_y = train_x.iloc[trn_idx],train_y.iloc[trn_idx]
        va_x,va_y = train_x.iloc[val_idx],train_y.iloc[val_idx]
        
        
        xg_train = xgb.DMatrix(tr_x, label=tr_y)
        xg_val = xgb.DMatrix(va_x,label=va_y)
        
        watchlist = [ (xg_train,'train'), (xg_val, 'validation') ]
        
        num_round = 1000
        bst = xgb.train(param, xg_train, num_round, watchlist ,verbose_eval=100,early_stopping_rounds=100);
        
        
        # get prediction
        oof[val_idx] = bst.predict( xgb.DMatrix(va_x) )
    #     prediction_xgb += bst.predict( test_x )/5

        predictions = bst.predict( xgb.DMatrix(test_x) )/5 +predictions
        
#         print(np.corrcoef(predictions[:,0],tmp_pre[:,0] ) )  ##比较不同种子下相似度 0.991 lgb模型相似度可能高些

#         tmp_pre = bst.predict( xgb.DMatrix(test_x) )


#         sub = test[['Group']]
#         prob_cols = [i for i in submit.columns if i not in ['Group']]
#         for i, f in enumerate(prob_cols):
#             sub[f] = predictions[:, i]
#         sub1 = pd.concat([sub,sub1],sort = False)    

print('logloss',log_loss(pd.get_dummies(train_y).values, oof))
print('ac',accuracy_score(train_y, np.argmax(oof,axis=1)))
print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[train_y] - oof))/480))

fold n 1
[0]	train-merror:0.347594	validation-merror:0.484552
Multiple eval metrics have been passed: 'validation-merror' will be used for early stopping.

Will train until validation-merror hasn't improved in 100 rounds.
[100]	train-merror:0.258665	validation-merror:0.44864
[200]	train-merror:0.227306	validation-merror:0.448376
Stopping. Best iteration:
[106]	train-merror:0.257477	validation-merror:0.446792

fold n 2
[0]	train-merror:0.352083	validation-merror:0.467917
Multiple eval metrics have been passed: 'validation-merror' will be used for early stopping.

Will train until validation-merror hasn't improved in 100 rounds.
[100]	train-merror:0.267578	validation-merror:0.44019
Stopping. Best iteration:
[32]	train-merror:0.292071	validation-merror:0.434645

fold n 3
[0]	train-merror:0.344491	validation-merror:0.494851
Multiple eval metrics have been passed: 'validation-merror' will be used for early stopping.

Will train until validation-merror hasn't improved in 100 rounds.
[100]	tr

In [ ]:
# 思路：线下随机构造groupid，然后获得线下验证策略。
def gen_sample(data, group_values, seed=0):
    group_values = shuffle(group_values, random_state=seed)
    # Group id 由group的值，做打乱获得，这里保存seed 信息，方便复现，因为有seed的话，就可以复现指定group
    data['Group'] = seed * 1000 + group_values
    return data


group_values = test.Group.values.copy()
data_2 = predictions.copy()
# 随机10组验证，道理上组数越多方差越小，越稳定
#for i in range(1, 10):
#    data_2 = pd.concat([
#        gen_sample(data[~data.label.isnull()], group_values, i),
#        data_2], ignore_index=True
#    )
#    print(i, data_2.shape)

#data_3 = data_2.groupby(['Group'], as_index=False)[bin_label + pred_label].mean()
#print(
#    'score:',
#    1 / (1 + 10 * abs(
#        data_3[data_3.Group >= 120][pred_label].values - data_3[data_3.Group >= 120][bin_label].values).mean())
#)

In [36]:
predictions

array([[0.28259293, 0.25881191, 0.27303635, 0.18555883],
       [0.29387285, 0.2469029 , 0.27838651, 0.18083774],
       [0.27385631, 0.25872677, 0.27197799, 0.19543894],
       ...,
       [0.20395142, 0.3866783 , 0.19433199, 0.21503829],
       [0.18923404, 0.35260142, 0.20113335, 0.25703119],
       [0.22100769, 0.34876471, 0.26775216, 0.16247545]])

In [38]:
test_sub = pd.read_csv('../data_3/second_round_testing_data.csv')
submit = pd.read_csv('../data_3/submit_example2.csv')
sub = test_sub[['Group']]
prob_cols = [i for i in submit.columns if i not in ['Group']]

for i, f in enumerate(prob_cols):
    sub[f] = predictions[:, i]
for i in prob_cols:
    sub[i] = sub.groupby('Group')[i].transform('mean')
sub = sub.drop_duplicates()
# sub.to_csv("result_3/xgb_8.csv",index=False)

In [40]:
sub.to_csv("result_3/xgb_8.csv",index=False)

In [37]:
for i, f in enumerate(prob_cols):
    sub1[f+'_rank'] = sub1.groupby('Group')[f].rank(method='first')
    sub1[f+'_rank'] = sub1[f+'_rank']-125
    sub1[f+'_rank'] = sub1[f+'_rank'].apply(lambda x: -(0.8/125/125/250)*x*x+1.4/250 )
    sub1[f+'_rank'] = sub1[f+'_rank']  #*sub['gcnt']
    sub1[f] = sub1[f]*sub1[f+'_rank']
for i in prob_cols:
    sub1[i] = sub1.groupby('Group')[i].transform('sum')  
sub1 = sub1[list(submit)].drop_duplicates().reset_index(drop = True)

In [12]:
sub1['all_sum'] = sub1[prob_cols].sum(axis=1)
for i in prob_cols:
    sub1[i] = sub1[i]/sub1['all_sum']
sub1['Group'] = submit['Group']
sub1[list(submit)].to_csv('result_3/xgb_3.csv',index=False) ###0.70104530000 


In [35]:
sub1.shape

(0, 0)

# 无限过过拟合训练

In [8]:
tmp_pre = np.zeros((len(test_x),4))
sub2 = pd.DataFrame()

xg_train = xgb.DMatrix( train_x, label=train_y)
train_x = xgb.DMatrix(train_x)
test_x = xgb.DMatrix(test_x)
# setup parameters for xgboost
for rand_seed in [500,1212,2019,51421,93693]:
    param = {
        'booster': 'gbtree',
        'objective': 'multi:softprob',  # 多分类的问题
        'num_class': 4,               # 类别数，与 multisoftmax 并用
        'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
        'max_depth': 12,               # 构建树的深度，越大越容易过拟合
        'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
        'subsample': 0.9,              # 随机采样训练样本
        'colsample_bytree': 0.9,       # 生成树时进行的列采样
        'min_child_weight': 3,
        'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
        'eta': 0.007,                  # 如同学习率
        'seed': rand_seed,
        'nthread': 4,                  # cpu 线程数
    }

    watchlist = [ (xg_train,'train'), (xg_train, 'train') ]
    num_round = 1200
    bst = xgb.train(param, xg_train, num_round, watchlist ,verbose_eval=100);
    # get prediction
    oof_xgb = bst.predict(train_x)
#     prediction_xgb += bst.predict( test_x )/5

    prediction = bst.predict( test_x )
    print(np.corrcoef(prediction[:,0],tmp_pre[:,0] ) )  ##比较不同种子下相似度 0.991 lgb模型相似度可能高些
    
    tmp_pre = bst.predict( test_x )

    
    sub = test[['Group']]
    prob_cols = [i for i in submit.columns if i not in ['Group']]
    for i, f in enumerate(prob_cols):
        sub[f] = prediction[:, i]
    sub2 = pd.concat([sub,sub2],sort = False)    
    
    print('logloss',log_loss(pd.get_dummies(train_y).values, oof_xgb))
    print('ac',accuracy_score(train_y, np.argmax(oof_xgb,axis=1)))
    print('mae',1/(1 + np.sum(np.absolute(np.eye(4)[train_y] - oof_xgb))/480))

[0]	train-merror:0.351009	train-merror:0.351009
[100]	train-merror:0.272578	train-merror:0.272578
[200]	train-merror:0.242421	train-merror:0.242421
[300]	train-merror:0.214799	train-merror:0.214799
[400]	train-merror:0.190715	train-merror:0.190715
[500]	train-merror:0.172758	train-merror:0.172758
[600]	train-merror:0.159079	train-merror:0.159079
[700]	train-merror:0.147565	train-merror:0.147565
[800]	train-merror:0.136633	train-merror:0.136633
[900]	train-merror:0.126439	train-merror:0.126439
[1000]	train-merror:0.116087	train-merror:0.116087
[1100]	train-merror:0.107267	train-merror:0.107267
[1199]	train-merror:0.098764	train-merror:0.098764
[[ 1. nan]
 [nan nan]]
logloss 0.5252485757488436
ac 0.9012358719763388
mae 0.03257245705491214
[0]	train-merror:0.353227	train-merror:0.353227
[100]	train-merror:0.273371	train-merror:0.273371
[200]	train-merror:0.242738	train-merror:0.242738
[300]	train-merror:0.215644	train-merror:0.215644
[400]	train-merror:0.190768	train-merror:0.190768
[500]

In [9]:
sub2.shape

(30000, 5)

In [10]:
for i, f in enumerate(prob_cols):
    sub2[f+'_rank'] = sub2.groupby('Group')[f].rank(method='first')
    sub2[f+'_rank'] = sub2[f+'_rank']-125
    sub2[f+'_rank'] = sub2[f+'_rank'].apply(lambda x: -(0.8/125/125/250)*x*x+1.4/250 )
    sub2[f+'_rank'] = sub2[f+'_rank']  #*sub['gcnt']
    sub2[f] = sub2[f]*sub2[f+'_rank']
for i in prob_cols:
    sub2[i] = sub2.groupby('Group')[i].transform('sum')  
sub2 = sub2[list(submit)].drop_duplicates().reset_index(drop = True)


In [11]:
sub2['all_sum'] = sub2[prob_cols].sum(axis=1)
for i in prob_cols:
    sub2[i] = sub2[i]/sub2['all_sum']
sub2['Group'] = submit['Group']
sub2[list(submit)].to_csv('result_3/xgb_12.csv',index=False) ###0.70104530000 


In [22]:
test_sub = pd.read_csv('../data/first_round_testing_data.csv')
submit = pd.read_csv('../data/submit_example.csv')
sub = test_sub[['Group']]
prob_cols = [i for i in submit.columns if i not in ['Group']]

for i, f in enumerate(prob_cols):
    sub[f] = prediction_xgb[:, i]
for i in prob_cols:
    sub[i] = sub.groupby('Group')[i].transform('mean')
sub = sub.drop_duplicates()
sub.to_csv("result/xgb_23.csv",index=False)

In [4]:
test_sub = pd.read_csv('../data/first_round_testing_data.csv')
submit = pd.read_csv('../data/submit_example.csv')
sub = test_sub[['Group']]

prob_cols = [i for i in submit.columns if i not in ['Group']]

for i, f in enumerate(prob_cols):
    sub[f] = prediction_xgb[:, i]

In [9]:
sub.to_csv('xgb_0.700.csv',index = False)